# Projeto Final - IA
## Previsão de time mandante vencer na Série A do Campeonato Brasileiro

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [ ]:
dataset = pd.read_csv("campeonato-brasileiro-full.csv", index_col = 0)
dataset.head()

,rodada,data,hora,mandante,visitante,formacao_mandante,formacao_visitante,tecnico_mandante,tecnico_visitante,vencedor,arena,mandante_Placar,visitante_Placar,mandante_Estado,visitante_Estado
ID,,,,,,,,,,,,,,,
1,1,29/3/2003,16:00,Guarani,Vasco,NaN,NaN,NaN,NaN,Guarani,Brinco de Ouro,4,2,SP,RJ
2,1,29/3/2003,16:00,Athletico-PR,Gremio,NaN,NaN,NaN,NaN,Athletico-PR,Arena da Baixada,2,0,PR,RS
3,1,30/3/2003,16:00,Flamengo,Coritiba,NaN,NaN,NaN,NaN,-,Maracanã,1,1,RJ,PR
4,1,30/3/2003,16:00,Goias,Paysandu,NaN,NaN,NaN,NaN,-,Serra Dourada,2,2,GO,PA
5,1,30/3/2003,16:00,Internacional,Ponte Preta,NaN,NaN,NaN,NaN,-,Beira Rio,1,1,RS,SP


In [ ]:
dataset.shape

(8025, 15)

In [ ]:
dataset["mandante"].value_counts()

Sao Paulo          390
Santos             390
Fluminense         390
Flamengo           389
Internacional      371
Athletico-PR       371
Corinthians        371
Atletico-MG        371
Gremio             350
Palmeiras          348
Cruzeiro           333
Botafogo-RJ        330
Vasco              295
Goias              276
Coritiba           276
Figueirense        219
Sport              209
Vitoria            198
Bahia              194
Ponte Preta        181
Juventude          143
Fortaleza          139
Avai               133
Ceara              133
Atletico-GO        133
Chapecoense        132
Parana             124
America-MG          95
Nautico             95
Sao Caetano         86
Criciuma            84
Paysandu            67
Guarani             65
Portuguesa          57
Bragantino          57
Santa Cruz          38
Cuiaba              38
Brasiliense         21
Santo Andre         19
Barueri             19
Ipatinga            19
America-RN          19
Gremio Prudente     19
Joinville  

In [ ]:
dataset["data"] = pd.to_datetime(dataset["data"])
dataset["cod_visitante"] = dataset["visitante"].astype("category").cat.codes
dataset["cod_mandante"] = dataset["mandante"].astype("category").cat.codes
dataset["time"] = 0 
dataset["hora_int"] = dataset["hora"].str.replace(":.+", "", regex=True).astype("int")
dataset["cod_dia"] = dataset["data"].dt.dayofweek

In [ ]:
for i in range(1, 8026):
  if dataset.at[i, "mandante"] == dataset.at[i, "vencedor"]:
    dataset.at[i, "vencedor"] = 1
  else:
    dataset.at[i, "vencedor"] = 0

In [ ]:
for i in range(0, 8026):
  for j in range(1, 8026-i):
    if i == dataset.at[j, "cod_visitante"]:
      dataset.at[j, "time"] = i

In [ ]:
dataset["cod_vencedor"] = dataset["vencedor"].astype("category").cat.codes

In [ ]:
dataset[["mandante", "visitante", "time", "cod_mandante", "vencedor"]].sort_values(by=["cod_mandante", "time"])

,mandante,visitante,time,cod_mandante,vencedor
ID,,,,,
3834,America-MG,Athletico-PR,2,0,1
5686,America-MG,Athletico-PR,2,0,1
6212,America-MG,Athletico-PR,2,0,1
7456,America-MG,Athletico-PR,2,0,1
7668,America-MG,Athletico-PR,2,0,1
...,...,...,...,...,...
2511,Vitoria,Vasco,43,44,1
3402,Vitoria,Vasco,43,44,1
4250,Vitoria,Vasco,43,44,1


In [ ]:
rf = RandomForestClassifier(n_estimators=100, min_samples_split=2, random_state=1)

In [ ]:
train = dataset[dataset["data"]<"2022-01-01"]

In [ ]:
train.shape

(7645, 21)

In [ ]:
test = dataset[dataset["data"]>"2022-01-01"]

In [ ]:
test.shape

(380, 21)

In [ ]:
predictors = ["cod_mandante", "cod_visitante", "hora_int", "cod_dia"]

In [ ]:
rf.fit(train[predictors], train["cod_vencedor"])

RandomForestClassifier(random_state=1)

In [ ]:
preds = rf.predict(test[predictors])

In [ ]:
acc = accuracy_score(test["cod_vencedor"], preds)
acc

0.5131578947368421

In [ ]:
combined = pd.DataFrame(dict(actual=test["cod_vencedor"], prediction = preds))

In [ ]:
pd.crosstab(index=combined["actual"], columns = combined["prediction"])

prediction,0,1
actual,,
0,110,102
1,83,85


In [ ]:
precision_score(test["cod_vencedor"], preds)

0.45454545454545453

In [ ]:
agrupar_partidas = dataset.groupby("mandante")

In [ ]:
grupo = agrupar_partidas.get_group("Botafogo-RJ")

In [ ]:
grupo

,rodada,data,hora,mandante,visitante,formacao_mandante,formacao_visitante,tecnico_mandante,tecnico_visitante,vencedor,...,mandante_Placar,visitante_Placar,mandante_Estado,visitante_Estado,cod_visitante,cod_mandante,time,hora_int,cod_dia,cod_vencedor
ID,,,,,,,,,,,,,,,,,,,,,
553,1,2004-04-21,16:00,Botafogo-RJ,Goias,NaN,NaN,NaN,NaN,0,...,1,4,RJ,GO,23,8,23,16,2,0
577,3,2004-04-28,20:30,Botafogo-RJ,Atletico-MG,NaN,NaN,NaN,NaN,0,...,2,2,RJ,MG,4,8,4,20,2,0
609,5,2004-09-05,16:00,Botafogo-RJ,Paysandu,NaN,NaN,NaN,NaN,0,...,2,2,RJ,PA,34,8,34,16,6,0
625,7,2004-05-22,16:00,Botafogo-RJ,Figueirense,NaN,NaN,NaN,NaN,0,...,1,1,RJ,SC,19,8,19,16,5,0
667,10,2004-06-20,16:00,Botafogo-RJ,Ponte Preta,NaN,NaN,NaN,NaN,0,...,0,1,RJ,SP,35,8,35,16,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7934,29,2022-03-10,20:00,Botafogo-RJ,Palmeiras,4-2-3-1,4-2-3-1,L. Castro,A. Moreira Ferreira,0,...,1,3,RJ,SP,32,8,32,20,3,0
7961,32,2022-10-16,18:00,Botafogo-RJ,Internacional,4-2-3-1,4-2-3-1,L. Castro,L. Venker de Menezes,0,...,0,1,RJ,RS,27,8,27,18,6,0
7978,34,2022-10-26,19:30,Botafogo-RJ,Bragantino,4-2-3-1,4-2-3-1,L. Castro,M. Nogueira Barbieri,1,...,2,1,RJ,SP,9,8,9,19,2,1


In [ ]:
def rolling_averages(grupo, cols, new_cols):
  grupo = grupo.sort_values("data")
  rolling_stats = grupo[cols].rolling(3, closed="left").mean()
  grupo[new_cols] = rolling_stats
  grupo = grupo.dropna(subset=new_cols)
  return grupo

In [ ]:
cols = ["mandante_Placar", "visitante_Placar"]
new_cols = [f"{c}_rolling" for c in cols]

In [ ]:
new_cols

['mandante_Placar_rolling', 'visitante_Placar_rolling']

In [ ]:
rolling_averages(grupo, cols, new_cols)

,rodada,data,hora,mandante,visitante,formacao_mandante,formacao_visitante,tecnico_mandante,tecnico_visitante,vencedor,...,mandante_Estado,visitante_Estado,cod_visitante,cod_mandante,time,hora_int,cod_dia,cod_vencedor,mandante_Placar_rolling,visitante_Placar_rolling
ID,,,,,,,,,,,,,,,,,,,,,
625,7,2004-05-22,16:00,Botafogo-RJ,Figueirense,NaN,NaN,NaN,NaN,0,...,RJ,SC,19,8,19,16,5,0,1.000000,2.333333
667,10,2004-06-20,16:00,Botafogo-RJ,Ponte Preta,NaN,NaN,NaN,NaN,0,...,RJ,SP,35,8,35,16,6,0,1.000000,1.333333
679,11,2004-06-27,16:00,Botafogo-RJ,Fluminense,NaN,NaN,NaN,NaN,0,...,RJ,RJ,21,8,21,16,6,0,0.333333,1.000000
701,13,2004-07-07,20:30,Botafogo-RJ,Internacional,NaN,NaN,NaN,NaN,1,...,RJ,RS,27,8,27,20,2,1,1.333333,1.666667
972,35,2004-07-10,20:30,Botafogo-RJ,Sao Caetano,NaN,NaN,NaN,NaN,0,...,RJ,SP,40,8,40,20,5,0,2.666667,1.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7650,1,2022-10-04,16:00,Botafogo-RJ,Corinthians,4-3-3,4-3-3,R. Oliveira,V. de Oliveira Lopes Pereira,0,...,RJ,SP,14,8,14,16,1,0,0.666667,0.333333
8014,37,2022-10-11,20:02,Botafogo-RJ,Santos,4-2-3-1,4-2-3-1,L. Castro,O. Ribeiro,1,...,RJ,SP,39,8,0,20,1,1,1.000000,1.333333
7961,32,2022-10-16,18:00,Botafogo-RJ,Internacional,4-2-3-1,4-2-3-1,L. Castro,L. Venker de Menezes,0,...,RJ,RS,27,8,27,18,6,0,2.000000,1.000000


In [ ]:
partidas_rolling = dataset.groupby("mandante").apply(lambda x: rolling_averages(x, cols, new_cols))

In [ ]:
partidas_rolling

rodada       data   hora    mandante     visitante  \
mandante   ID                                                         
America-MG 3791      33 2011-06-11  17:00  America-MG   Corinthians   
           3508       5 2011-06-18  21:00  America-MG      Cruzeiro   
           3529       7 2011-06-29  19:30  America-MG      Flamengo   
           3545       8 2011-07-07  21:00  America-MG     Palmeiras   
           3613      15 2011-07-08  16:00  America-MG    Fluminense   
...                 ...        ...    ...         ...           ...   
Vitoria    6420      30 2018-10-21  16:00     Vitoria   Corinthians   
           6426      31 2018-10-26  19:31     Vitoria     Sao Paulo   
           6450      33 2018-11-11  16:59     Vitoria         Bahia   
           6467      35 2018-11-17  18:59     Vitoria  Athletico-PR   
           6489      37 2018-11-25  17:01     Vitoria        Gremio   

                formacao_mandante formacao_visitante tecnico_mandante  \
mandante   ID                                                           
America-MG 3791               NaN                NaN              NaN   
           3508               NaN                NaN              NaN   
           3529               NaN                NaN              NaN   
           3545               NaN                NaN              NaN   
           3613               NaN                NaN              NaN   
...                           ...                ...              ...   
Vitoria    6420           4-1-4-1            4-2-3-1    P. Carpegiani   
           6426           4-1-4-1              4-4-2    P. Carpegiani   
           6450           4-2-3-1            4-2-3-1         J. Burse   
           6467             4-3-3            4-2-3-1         J. Burse   
           6489           4-1-4-1            4-2-3-1         J. Burse   

                  tecnico_visitante vencedor  ... mandante_Estado  \
mandante   ID                                 ...                   
America-MG 3791                 NaN        1  ...              MG   
           3508                 NaN        0  ...              MG   
           3529                 NaN        0  ...              MG   
           3545                 NaN        0  ...              MG   
           3613                 NaN        1  ...              MG   
...                             ...      ...  ...             ...   
Vitoria    6420  J. Ribeiro Ventura        0  ...              BA   
           6426  D. Aguirre Camblor        0  ...              BA   
           6450    E. Alves Moreira        0  ...              BA   
           6467   T. Retzalff Nunes        0  ...              BA   
           6489       Renato Gaúcho        0  ...              BA   

                 visitante_Estado  cod_visitante cod_mandante time  hora_int  \
mandante   ID                                                                  
America-MG 3791                SP             14            0   14        17   
           3508                MG             17            0   17        21   
           3529                RJ             20            0   20        19   
           3545                SP             32            0   32        21   
           3613                RJ             21            0   21        16   
...                           ...            ...          ...  ...       ...   
Vitoria    6420                SP             14           44   14        16   
           6426                SP             41           44   41        19   
           6450                BA              6           44    6        16   
           6467                PR              2           44    2        18   
           6489                RS             24           44   24        17   

                 cod_dia  cod_vencedor  mandante_Placar_rolling  \
mandante   ID                                                     
America-MG 3791        5             1                 2.666667   
   

In [ ]:
partidas_rolling.index = range(partidas_rolling.shape[0])

In [ ]:
def predicoes(data, predictors):
  train = data[data["data"]<"2022-01-01"]
  test = data[data["data"]>"2022-01-01"]
  rf.fit(train[predictors], train["cod_vencedor"])
  preds = rf.predict(test[predictors])
  combined = pd.DataFrame(dict(atual = test["cod_vencedor"], predicted=preds), index=test.index)
  precision = precision_score(test["cod_vencedor"], preds)
  return combined, precision

In [ ]:
combined, precision = predicoes(partidas_rolling, predictors + new_cols)

In [ ]:
acc = accuracy_score(test["cod_vencedor"], preds)
acc

0.5131578947368421

In [ ]:
precision

0.5297297297297298

In [ ]:
combined

,atual,predicted
73,1,0
74,1,1
75,1,1
76,1,0
77,1,0
...,...,...
7192,1,1
7193,1,1
7194,1,1
7195,0,0


In [ ]:
combined = combined.merge(partidas_rolling[["data", "mandante", "visitante", "cod_vencedor"]], left_index=True, right_index=True)

In [ ]:
combined

,atual,predicted,data,mandante,visitante,cod_vencedor
73,1,0,2022-02-11,America-MG,Internacional,1
74,1,1,2022-03-07,America-MG,Goias,1
75,1,1,2022-03-09,America-MG,Coritiba,1
76,1,0,2022-04-06,America-MG,Cuiaba,1
77,1,0,2022-04-16,America-MG,Juventude,1
...,...,...,...,...,...,...
7192,1,1,2022-10-04,Sao Paulo,Athletico-PR,1
7193,1,1,2022-10-20,Sao Paulo,Coritiba,1
7194,1,1,2022-10-27,Sao Paulo,Atletico-GO,1
7195,0,0,2022-11-09,Sao Paulo,Corinthians,0


In [ ]:
agrupar_partidas = combined.groupby("mandante")
grupo = agrupar_partidas.get_group("Flamengo")
grupo

,atual,predicted,data,mandante,visitante,cod_vencedor
3655,1,1,2022-01-10,Flamengo,Bragantino,1
3656,0,0,2022-02-11,Flamengo,Corinthians,0
3657,0,1,2022-04-09,Flamengo,Ceara,0
3658,1,1,2022-04-17,Flamengo,Sao Paulo,1
3659,0,1,2022-04-20,Flamengo,Palmeiras,0
3660,0,1,2022-05-06,Flamengo,Fortaleza,0
3661,0,0,2022-05-10,Flamengo,Internacional,0
3662,1,0,2022-05-21,Flamengo,Goias,1
3663,1,1,2022-06-15,Flamengo,Cuiaba,1
3664,1,1,2022-06-25,Flamengo,America-MG,1


In [ ]:
j=0
z=0
for i in range(3655, 3674):
  if grupo.at[i, "predicted"] == grupo.at[i, "cod_vencedor"]:
    j = j + 1
  z= z+1
print(j/z)

0.6842105263157895
